In [ ]:
# 安裝富果行情 API 套件
# !pip install fugle-marketdata -U -q
# 更改時區用
# !pip install pytz
# 定時執行用
# !pip install schedule

# 安裝 linetool 套件
# !pip install linetool -q

# 載入相關套件
# !pip install fugle_realtime

In [ ]:
#安裝指標函式庫
url = 'https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h166bdaf_1.tar.bz2'
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = 'https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py310hde88566_4.tar.bz2'
!curl -L $url | tar xj -C /usr/local/lib/python3.10/dist-packages/ lib/python3.10/site-packages/talib --strip-components=3

# 載入相關套件
import matplotlib.pyplot as plt
from datetime import timedelta
import pandas as pd
import numpy as np
import requests
import json
import time
from configparser import ConfigParser
from fugle_realtime import (HttpClient, WebSocketClient)


# 載入套件
import talib
from fugle_marketdata import WebSocketClient, RestClient
import pandas as pd

  Preparing metadata (setup.py) ... done
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4445    0  4445    0     0  11827      0 --:--:-- --:--:-- --:--:-- 11853
100  517k  100  517k    0     0   473k      0  0:00:01  0:00:01 --:--:-- 1695k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4509    0  4509    0     0  17655      0 --:--:-- --:--:-- --:--:-- 17682
100  392k  100  392k    0     0   429k      0 --:--:-- --:--:-- --:--:-- 1075k


In [ ]:
# API Key 可至金鑰申請與管理頁的行情 API 區申請：https://developer.fugle.tw/docs/key/
client = RestClient(api_key = 'OGNkNGViMDUtZWE2Ni00YzE2LTk4MTctODg1YzBlZTM0MGVkIGI4YjA2NjExLWM5ODAtNDk0NC1iZTZiLWIwYzU4MjI4Zjc4ZQ==')
stock = client.stock  # Stock REST API client

# macd_timely() 即時策略監測


In [ ]:
import pandas as pd
from datetime import datetime

import talib
import pandas as pd
import time
from fugle_marketdata import RestClient
from datetime import datetime
import pytz

api_key = 'OGNkNGViMDUtZWE2Ni00YzE2LTk4MTctODg1YzBlZTM0MGVkIGI4YjA2NjExLWM5ODAtNDk0NC1iZTZiLWIwYzU4MjI4Zjc4ZQ=='
client = RestClient(api_key=api_key)
stock = client.stock

# 設置台北時區
taipei_tz = pytz.timezone('Asia/Taipei')




def macd_timely():
  # 獲取股票數據
  stock_latest = stock.intraday.candles(symbol='2330', timeframe='1')
  stock_data = pd.DataFrame(stock_latest['data'])

  # 建立一個macd_signal空dataframe
  columns = ['Signal', 'DIFF', 'DEA', 'macd']
  macd_signal = pd.DataFrame(columns=columns)



  # 獲取當前的台北時間
  taipei_time = datetime.now(taipei_tz)
  time_message = taipei_time.strftime("台北時間: %Y-%m-%d %H:%M:%S")

  #再用ta-lib函式庫去做macd指標
  # 計算 MACD
  macd, signal, hist = talib.MACD(stock_data['close'], fastperiod=5, slowperiod=10, signalperiod=9)

  macd_signal['Signal'] = "0"
  macd_signal['DIFF'] = macd #macd對應diff
  macd_signal['DEA'] = signal #macdsignal（对应dea）
  macd_signal['macd'] = hist



  #凡事只要檢查最新的即可(活在當下)
  i = len(macd_signal)-1
  j = len(stock_data)-1

  if macd_signal['DIFF'][i] > macd_signal['DEA'][i] and macd_signal['DIFF'][i-1] <= macd_signal['DEA'][i-1] and macd_signal['DIFF'][i] > 0 and macd_signal['DEA'][i] > 0:

      macd_signal['Signal'][i] = '黃金交叉出現!'
      return_message = "目前收盤價:"+ str(stock_data['close'][j])+'黃金交叉出現!'
      combined_message = f"{time_message},{return_message}"
      #print(macd_signal['DIFF'][i], macd_signal['DEA'][i],"黃金交叉出現!")


  elif macd_signal['DIFF'][i] < macd_signal['DEA'][i] and macd_signal['DIFF'][i-1] >= macd_signal['DEA'][i-1] and macd_signal['DIFF'][i] < 0 and macd_signal['DEA'][i] < 0:

      macd_signal['Signal'][i] = '死亡交叉出沒><!'
      return_message = "目前收盤價:"+ str(stock_data['close'][j])+'死亡交叉出現!'
      combined_message = f"{time_message},{return_message}"
      print(macd_signal['DIFF'][i], macd_signal['DEA'][i],"死亡交叉出沒><!")
  else:
      return_message = "目前收盤價:"+ str(stock_data['close'][j])+" 目前macd這邊沒事"
      combined_message = f"{time_message},{return_message}"

  return combined_message

In [ ]:
#testing
print(macd_timely())

台北時間: 2023-12-23 21:41:26,目前收盤價:582黃金交叉出現!


<ipython-input-3-6ce8aa0ad9ae>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  macd_signal['Signal'][i] = '黃金交叉出現!'


In [ ]:
def bollinger_bands_timely():
    # 獲取股票數據
    stock_latest = stock.intraday.candles(symbol='2330', timeframe='1')
    stock_data = pd.DataFrame(stock_latest['data'])

    # 建立一個布林通道上軌的空 dataframe
    columns = ['Upper', 'Middle', 'Lower', 'Close']
    bbands_signal = pd.DataFrame(columns=columns)

    # 獲取當前的台北時間
    taipei_time = datetime.now(taipei_tz)
    time_message = taipei_time.strftime("台北時間: %Y-%m-%d %H:%M:%S")

    # 計算布林通道
    upper, middle, lower = talib.BBANDS(stock_data['close'], timeperiod=20, nbdevup=2, nbdevdn=2)

    bbands_signal['Upper'] = upper
    bbands_signal['Middle'] = middle
    bbands_signal['Lower'] = lower
    bbands_signal['Close'] = stock_data['close']

    # 檢查最新的值
    i = len(bbands_signal) - 1
    j = len(stock_data) - 1

    if stock_data['close'][j] > bbands_signal['Upper'][i]:
        signal_message = '價格超過布林通道上軌！'
    elif stock_data['close'][j] < bbands_signal['Lower'][i]:
        signal_message = '價格低於布林通道下軌！'
    else:
        signal_message = '價格在布林通道範圍內。'

    return_message = signal_message
    return return_message


In [ ]:
#testing
print(bollinger_bands_timely())

價格超過布林通道上軌！


In [ ]:
def rsi_timely():
    # 獲取股票數據
    stock_latest = stock.intraday.candles(symbol='2330', timeframe='1')
    stock_data = pd.DataFrame(stock_latest['data'])
    # 計算 RSI
    rsi_period = 14
    stock_data['rsi'] = talib.RSI(stock_data['close'], timeperiod=rsi_period)

    # 獲取當前的台北時間
    taipei_time = datetime.now(taipei_tz)
    time_message = taipei_time.strftime("台北時間: %Y-%m-%d %H:%M:%S")

    # 檢查 RSI 信號
    i = len(stock_data) - 1
    j = len(stock_data) - 1

    if stock_data['rsi'][i] > 70:
        signal_message = '超買信號！'
    elif stock_data['rsi'][i] < 30:
        signal_message = '超賣信號！'
    else:
        signal_message = '目前 RSI 無信號變動。'


    return signal_message


In [ ]:
#testing
print(rsi_timely())

目前 RSI 無信號變動。


## 🎁三策略融合

In [ ]:
def tripleStrategy():
  macd = macd_timely()
  rsi = rsi_timely()
  boollinger = bollinger_bands_timely()
  final_signal = macd + "\n" + rsi +"\n"+ boollinger
  return final_signal

In [ ]:
#testing
print(tripleStrategy())

<ipython-input-3-6ce8aa0ad9ae>:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  macd_signal['Signal'][i] = '死亡交叉出沒><!'


-0.12385451067189024 -0.039058600397439115 死亡交叉出沒><!
台北時間: 2023-12-28 09:59:22,目前收盤價:591死亡交叉出現!
目前 RSI 無信號變動。
價格低於布林通道下軌！


# Line notify串接

In [ ]:
from datetime import datetime, timedelta
import requests
import schedule
import time
import pytz

# LINE Notify 的 API Token
line_token = "rquRjXYZzcyyRJ2hfWdIwvaEKbMWJFdCanZgyCCACYi"

def send_line_notify(message):
    url = "https://notify-api.line.me/api/notify"
    headers = {
        "Authorization": "Bearer " + line_token
    }
    data = {"message": message}
    response = requests.post(url, headers=headers, data=data)
    return response

def task():
    tz_taipei = pytz.timezone('Asia/Taipei')
    end_time = datetime.now(tz_taipei) + timedelta(hours=4, minutes=30)
    while datetime.now(tz_taipei) < end_time:
        send_line_notify(tripleStrategy())
        time.sleep(60)  # 等待一分鐘

# 設定每天台北時間早上 9:00 執行 task 函數
schedule.every().day.at("02:02").do(task) # 台北時間早上 9:00 對應 UTC 時間 01:00(開盤)

while True:
    schedule.run_pending()
    time.sleep(1)


KeyboardInterrupt: ignored